**Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
import random

# Definindo a semente para reprodução
np.random.seed(42)
random.seed(42)

**Reading Files**

In [2]:
magic_items = pd.read_csv('data/magic_items.csv', sep=';')
adventure_gear = pd.read_csv('data/adventuring_gear.csv', sep=';')
armor = pd.read_csv('data/armor.csv', sep=';')
poisons = pd.read_csv('data/poisons.csv', sep=';')
potions = pd.read_csv('data/potions.csv', sep=';')
weapons = pd.read_csv('data/weapons.csv', sep=';')
names = pd.read_csv('data/names.csv', sep=';')

In [3]:
magic_items.head()

,item_id,Name,Price,Rarity,Category
0,001-ACo,Ammunition +1 (Per),15 gp,Uncommon,Consumable Items
1,002-ACo,Ammunition +2 (Per),50 gp,Rare,Consumable Items
2,003-ACo,Ammunition +3 (Per),250 gp,Very Rare,Consumable Items
3,004-ACo,Arrow of Slaying,400 gp,Very Rare,Consumable Items
4,005-BCo,Bead of Force,"1,000 gp",Rare,Consumable Items


In [4]:
adventure_gear.head()

,item_id,Name,Price,Weight,type
0,01-Ars,Abacus,2 gp,2 lb.,Others
1,02-Ars,Acid (vial),25 gp,1 lb.,Others
2,03-Ars,Alchemist's Fire (flask),50 gp,1 lb.,Others
3,04-Aon,Arrows (20),1 gp,1 lb.,Ammunition
4,05-Bon,Blowgun Needle (50),1 gp,1 lb.,Ammunition


In [5]:
weapons.head()

,item_id,Name,Price,Damage,Weight,Properties,type
0,01-Cns,Club,1 sp,1d4 Bludgeon,2 lb.,Light,Simple Melee Weapons
1,02-Dns,Dagger,2 gp,1d4 Piercing,1 lb.,"Finesse, Light, Thrown (20/60)",Simple Melee Weapons
2,03-Gns,Greatclub,2 sp,1d8 Bludgeon,10 lb.,Two-handed,Simple Melee Weapons
3,04-Hns,Handaxe,5 gp,1d6 Slashing,2 lb.,"Light, Thrown (20/60)",Simple Melee Weapons
4,05-Jns,Javelin,5 sp,1d6 Piercing,2 lb.,Thrown (30/120),Simple Melee Weapons


In [6]:
armor.head()

,item_id,Name,Price,AC,Weight,Requirements,Stealth,type
0,01-Por,Padded,5 gp,11 + Dex,8 lb.,NaN,Disadvantage,Light Armor
1,02-Lor,Leather,10 gp,11 + Dex,10 lb.,NaN,NaN,Light Armor
2,03-Sor,Studded Leather,45 gp,12 + Dex,13 lb.,NaN,NaN,Light Armor
3,04-Hor,Hide,10 gp,12 + Dex(max2),12 lb.,NaN,NaN,Medium Armor
4,05-Cor,Chain Shirt,50 gp,13 + Dex(max2),20 lb.,NaN,NaN,Medium Armor


In [7]:
potions.head()

,item_id,Name,Price,Rarity
0,01-Pon,Potion of Healing,50 gp,Common
1,02-Pon,Potion of Greater Healing,150 gp,Uncommon
2,03-Pre,Potion of Superior Healing,450 gp,Rare
3,04-Pre,Potion of Supreme Healing,"1,350 gp",Very Rare
4,05-Ere,Elixir of Health,120 gp,Rare


In [8]:
poisons.head()

,item_id,Name,Price,Type,DC
0,01-Aed,Assassin's blood,150 gp,Ingested,10.0
1,02-Ted,Truth serum,150 gp,Ingested,11.0
2,03-Cct,Carrion crawler mucus,200 gp,Contact,13.0
3,04-Dry,Drow poison,200 gp,Injury,13.0
4,05-Sry,Serpent venom,200 gp,Injury,11.0


****

# **Data Creation**

**Passos:**

1. Filtrar as Raridades dos itens
1. Criar uma tabela que contenha todos os produtos
    * A tabela deve ter apenas informações básicas sobre os itens, como: id | nome | preço | tipo.
1. Criar uma tabela com as informações dos clientes
1. Criar a tabela fato de **vendas**




### **Filtrar Raridades:**

In [9]:
magic_items['Rarity'].value_counts()

Rare         114
Uncommon      83
Very Rare     63
Legendary     38
Common         2
Name: Rarity, dtype: int64

Como eu pretendo simular as vendas de um vendedor comum, não vou usar itens que sejam: Very Rare, Legendary.

In [10]:
# função para filtrar a raridade dos items
def filter_rarity(dataframe):
    unwanted = ['Very Rare', 'Legendary']
    df = dataframe.copy()
    df = df.query("Rarity not in @unwanted")
    return df

In [11]:
magic_items_filtered = filter_rarity(magic_items)

In [12]:
magic_items_filtered['Rarity'].value_counts()

Rare        114
Uncommon     83
Common        2
Name: Rarity, dtype: int64

In [13]:
potions_filtered =  filter_rarity(potions)

In [14]:
potions_filtered['Rarity'].value_counts()

Uncommon    10
Rare        10
Common       2
Name: Rarity, dtype: int64

Nenhum outro Dataset tem a coluna 'Rarity'

### **Criar tabela Produtos**

Primeiramente devo mudar as colunas 'type' para 'cetegory', depois criar uma nova coluna 'type' com o tipo de item de cada dataset.

In [15]:
# função para substituir a coluna type e colocar os nomes das colunas em minúsculo
def replace_lower_column(df):
    df = df.columns.str.lower().str.replace('type', 'category')
    return df

In [16]:
adventure_gear.columns = replace_lower_column(adventure_gear)
magic_items_filtered.columns = replace_lower_column(magic_items_filtered)
armor.columns = replace_lower_column(armor)
weapons.columns = replace_lower_column(weapons)
potions_filtered.columns =  replace_lower_column(potions_filtered)
poisons.columns = replace_lower_column(poisons)

**Criando as novas colunas 'type':**

In [17]:
adventure_gear['type'] = 'adventure_gear'
magic_items_filtered['type'] = 'magic_item'
weapons['type'] = 'weapon'
potions_filtered['type'] = 'potion'
poisons['type'] = 'poison'
armor['type'] = 'armor'

In [18]:
magic_items_filtered

,item_id,name,price,rarity,category,type
0,001-ACo,Ammunition +1 (Per),15 gp,Uncommon,Consumable Items,magic_item
1,002-ACo,Ammunition +2 (Per),50 gp,Rare,Consumable Items,magic_item
4,005-BCo,Bead of Force,"1,000 gp",Rare,Consumable Items,magic_item
5,006-CCo,Chime of Opening,400 gp,Rare,Consumable Items,magic_item
6,007-DCo,Deck of Illusions,900 gp,Uncommon,Consumable Items,magic_item
...,...,...,...,...,...,...
291,292-ICo,Instrument of the Bards - Cli Lyre,"2,500 gp",Rare,Combat Items,magic_item
292,293-ICo,Instrument of the Bards - Fochulan Bandlore,"1,500 gp",Uncommon,Combat Items,magic_item
293,294-ICo,Instrument of the Bards - Mac-Fuirmidh Cittern,"2,000 gp",Uncommon,Combat Items,magic_item
294,295-ICo,Instrument of the Bards - Doss Lute,"1,500 gp",Uncommon,Combat Items,magic_item


**Criando a tabela Produtos:**

| id | nome | preço | tipo |


In [19]:
wanted_cols = ['item_id', 'name', 'price', 'type']

In [20]:
product = magic_items_filtered[wanted_cols].copy()

In [21]:
product = pd.concat([product,
                     adventure_gear[wanted_cols],
                     weapons[wanted_cols],
                     armor[wanted_cols],
                     potions_filtered[wanted_cols],
                     poisons[wanted_cols]],
                     ignore_index=True)

In [22]:
product['type'].value_counts()

magic_item        199
adventure_gear    109
weapon             37
potion             22
poison             16
armor              13
Name: type, dtype: int64

In [23]:
product.head()

,item_id,name,price,type
0,001-ACo,Ammunition +1 (Per),15 gp,magic_item
1,002-ACo,Ammunition +2 (Per),50 gp,magic_item
2,005-BCo,Bead of Force,"1,000 gp",magic_item
3,006-CCo,Chime of Opening,400 gp,magic_item
4,007-DCo,Deck of Illusions,900 gp,magic_item


In [39]:
# testando o relacionamento das tabelas
MI_itemID =  random.choices(product['item_id'].loc[product['type'] == 'magic_item'])
magic_items_filtered.query('item_id == @MI_itemID')

,item_id,name,price,rarity,category,type
27,028-NCo,Necklace of Fireballs (6 beads),"8,000 gp",Rare,Consumable Items,magic_item


O relacionamento está funcionando como o desejado.

## **Criar a Tabela Cliente**

**Atributos do cliente:**


* id
* nome
* sexo
* idade
* raça
* classe
* endereço (cidade) (talvez)
* contato (talvez)

In [25]:
# checando duplicatas
names.duplicated().sum()

45

In [26]:
names.drop_duplicates(inplace=True)

In [66]:
names.shape

(365, 2)

referência = https://bg3.wiki/wiki/Races

In [40]:
races = pd.read_csv('data/races.csv', sep=';')
races

,race,base_age,max_age,maximum_age_range
0,Dragonborn,15,80,80 + 1d20
1,Drow,80,225,225 + 3d100
2,Dwarf,50,350,300 + 2d100
3,Elf,90,750,425 + 5d100
4,Githyanki,30,250,250 + 1d100
5,Gnome,60,200,200 + 3d100
6,Half-Elf,15,125,125 + 3d20
7,Halfling,20,100,100 + 1d100
8,Half-Orc,12,60,90 + 2d20
9,Human,15,90,90 + 2d20


referência: https://www.dndbeyond.com/sources/basic-rules/classes#ClassesSummary

In [43]:
classes = pd.read_csv('data\classes.csv', sep=';')
classes

,Class,Armor,Weapon
0,Barbarian,"Light armor, medium armor, shields","Simple weapons, martial weapons"
1,Bard,Light armor,"simple weapons, hand crossbows, longswords, ra..."
2,Cleric,"Light armor, medium armor, shields",Simple weapons
3,Druid,"Light armor, medium armor, shields","Clubs, daggers, darts, javelins, maces, quarte..."
4,Fighter,"Light armor, medium armor, heavy armor, shields","Simple weapons, martial weapons"
5,Monk,None,"Simple weapons, shortswords"
6,Paladin,"Light armor, medium armor, heavy armor, shields","Simple weapons, martial weapons"
7,Ranger,"Light armor, medium armor, shields","Simple weapons, martial weapons"
8,Rogue,Light armor,"Simple weapons, hand crossbows, longswords, ra..."
9,Sorcerer,None,"Daggers, darts, slings, quarterstaffs, light c..."


A tabela de cliente deve ter a seguinte estrutura:

| customer_id| name    | sex      | race     | age     | class   |
| -------- | ------- | -------- | ------- | -------- | ------- |
| -------- |-------- | -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |

In [44]:
import time
import string

# criando função para gerar IDs
def generate_custom_id():
    """Function to generate random IDs"""
    timestamp = str(int(time.time()))
    random_chars = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
    return f"{timestamp}-{random_chars}"   # return new unique ID

In [48]:
# random ID
custom_id = generate_custom_id()
print(custom_id)

1701721688-DD4V30


In [235]:
# criando um dataframe com os randoms IDs
customers = pd.DataFrame({
    'customer_id': [generate_custom_id() for i in range(1, names.shape[0] + 1)]})
customers

,customer_id
0,1701729021-NW6L4M
1,1701729021-1TRPUY
2,1701729021-D6I5HO
3,1701729021-9CY9PE
4,1701729021-0HDJ7O
...,...
360,1701729021-JJ63XV
361,1701729021-FIUD99
362,1701729021-XRO37U
363,1701729021-9SSBU7


In [236]:
# checando IDs duplicados
customers.duplicated().sum()

0

In [234]:
names.isna().sum()

name    0
sex     0
dtype: int64

In [ ]:
# resetando os índices
names.reset_index(drop=True, inplace=True)

In [242]:
# inserindo os nomes e sexos na tabela de clientes
customers["name"] = names['name']
customers["sex"] = names['sex']

In [243]:
customers

,customer_id,name,sex
0,1701729021-NW6L4M,Veklani Daargen,female
1,1701729021-1TRPUY,Kasaki Wygarthe,female
2,1701729021-D6I5HO,Rosalyn Faringray,female
3,1701729021-9CY9PE,Atalya Webb,female
4,1701729021-0HDJ7O,Grenenzel Lyfalia,female
...,...,...,...
360,1701729021-JJ63XV,Brey Danamark,female
361,1701729021-FIUD99,Remora Lamoth,female
362,1701729021-XRO37U,Ronefel Irva,female
363,1701729021-9SSBU7,Sachil Redraven,female


In [245]:
races['race'].unique()

array(['Dragonborn', 'Drow', 'Dwarf', 'Elf', 'Githyanki', 'Gnome',
       'Half-Elf', 'Halfling', 'Half-Orc', 'Human', 'Tiefling'],
      dtype=object)

In [246]:
# limpando a coluna 'race'
races['race'] = races['race'].str.replace('\xa0', '').str.strip()

In [247]:
customers['race'] = [random.choice(list(races['race'])) for _ in range(names.shape[0])]

Agora vem uma etapa não tão simples, a idade, cada raça tem um período de vida diferente e pretendo conservar essas características também.

In [248]:
# criando a função que irá gerar a idade aleatória baseada da raça que foi dada
def random_age(X):
    """This function returns a random age based on the race"""
    race_infos = races[races['race'] == X].reset_index(drop=True)
    return random.randint(race_infos['base_age'][0], race_infos['max_age'][0])

In [249]:
customers['age'] = customers['race'].apply(random_age)

Agora resta apenas escolher as classes para cada cliente.

In [250]:
customers['class'] = [random.choice(list(classes['Class'])) for _ in range(customers.shape[0])]

Conferindo a estrutura da tabela:

| customer_id| name    | sex      | race     | age     | class   |
| -------- | ------- | -------- | ------- | -------- | ------- |
| -------- |-------- | -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |

In [251]:
customers.sample(n=10)

,customer_id,name,sex,race,age,class
46,1701729021-N1TCJF,Praxana Armanci,female,Elf,396,Barbarian
244,1701729021-OR6XN6,Ningyan Daargen,male,Gnome,115,Barbarian
191,1701729021-T30GCH,Eandro Targana,male,Tiefling,81,Cleric
169,1701729021-LLQLLD,Nilus Daargen,male,Dwarf,237,Bard
181,1701729021-E8683P,Quentin Ramcrown,male,Gnome,112,Paladin
194,1701729021-QAFQSD,Talfen Atalya,male,Githyanki,48,Rogue
249,1701729021-DO1DFS,Velene Hackshield,female,Dragonborn,43,Warlock
333,1701729021-2A2Q9O,Azura Stormchapel,female,Tiefling,47,Druid
107,1701729021-43F1AY,Fenton Seratolva,male,Half-Elf,33,Bard
5,1701729021-9X66D7,Venefiq Welfer,female,Half-Elf,25,Cleric


## **Criar a tabela de Vendas**